In [1]:
# #### Pre Installs
# import os
# os.environ['openAIAPIKey'] = ''
# os.environ['serpAPIKey'] = ''
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32009 sha256=7bb8663db5768909401dd697993b670de3e8d49cb0d4bc5e4f0b4375c8d673b1
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [2]:
appConfig = {
    "NoOfLinks" : 3
}

In [3]:
import os

class getApiKeys:

  def __init__(self):
    pass

  def getApiKeys(self,apiKey):
    return os.environ.get(apiKey)


In [22]:
import serpapi
import json
from serpapi import GoogleSearch # Add this import statement

class searchForLinks:
  def __init__(self):
    pass

  def getSearchResults(self,searchWord):
    searpApiKey = getApiKeys()
    params = {
      "q": searchWord,
      "hl": "en",
      "gl": "us",
      "google_domain": "google.com",
      "api_key": searpApiKey.getApiKeys('serpAPIKey')
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    linksList = []

    if 'organic_results' in results:
        for res in results['organic_results']:
            if 'link' in res:  # Check if 'link' key exists in each result dictionary
                linksList.append(res['link'])
    else:
        print("No organic results found in the response.")

    return linksList[:appConfig['NoOfLinks']]

In [25]:
sfl = searchForLinks()
sfl.getSearchResults('coffee')

['https://en.wikipedia.org/wiki/Coffee',
 'https://www.peets.com/',
 'https://www.starbucks.com/']

In [16]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display


class webCrawler:

  def __init__(self):
    pass

  def crawlresults(self,srch):
    sfl = searchForLinks()
    linktocrawl = sfl.getSearchResults(srch)
    print(linktocrawl)
    crawlJson = {}
    for links in linktocrawl:
      crawlJson.update(self.crawl(links))

    return crawlJson

  def crawl(self, url: str):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.title.string if soup.title else "Title not found"

    if soup.body:
      for irrelevant in soup.body(["script", "style","img","input"]):
        irrelevant.decompose()
      text = soup.body.get_text(separator="\n", strip=True)
    else:
      text = ""

    return {title : text}

    # links = [link.get('href') for link in soup.find_all('a')]
    # self.links = [link for link in links if link]

In [17]:
crawl = webCrawler()
crawl.crawlresults('Stock')

No organic results found in the response.
[]


{}